In [1]:
import pandas as pd
import warnings
from IPython.utils import io
import sys
import numpy as np
from functools import reduce

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

stars_dir = '~/GitHub/stars-data-builder/'
hos_dir = '~/Desktop/Rush/CMS_HospitalArchives/'

dates_df = pd.DataFrame(columns = ['Measure ID', 'Start Date', 'End Date'])

In [2]:
def curate(df):

    try:
        df = df[df['PROVIDER_ID'] != np.nan]
        df['PROVIDER_ID'] = df['PROVIDER_ID'].values.astype(str)
        
        ids = df['PROVIDER_ID'].tolist()
        ids2 = []
        for i in ids:
            if len(i) < 6:
                i = '0' + i
            ids2.append(i)
        df['PROVIDER_ID'] = ids2
        
    except:
        pass
    
    for c in list(df):    
        try:
            df[c] = df[c].str.replace("\t","")
        except:
            pass

    if 'Unnamed: 0' in list(df):
        df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
    return df

# Use 2023 SAS input file to get the column labels needed for 2024 prognostication

In [3]:
sas_input_df_2023 = pd.read_sas(stars_dir + 'Reproduce_Stars_Input/2023/Input_File/alldata_2023jul.sas7bdat', 
                           format = 'sas7bdat', encoding = "utf8")

sas_cols_2023 = list(sas_input_df_2023)

print(len(sas_cols_2023), 'columns in 2023 Stars SAS input file (not all get used by the SAS programs):')
print(sas_cols_2023, '\n')

print('Create columns for 2024 data')

sas_cols = list(sas_cols_2023)

sas_input_df_2023 = curate(sas_input_df_2023)
prvdrs_2023 = sas_input_df_2023['PROVIDER_ID'].unique().tolist()

print(len(prvdrs_2023), 'hospitals in 2023 Stars output file\n')
prvdrs_2023 = sorted(list(set(prvdrs_2023)))

print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')

sas_input_df_2023.head()

95 columns in 2023 Stars SAS input file (not all get used by the SAS programs):
['PROVIDER_ID', 'HAI_1_DEN_VOL', 'HAI_2_DEN_VOL', 'HAI_3_DEN_VOL', 'HAI_4_DEN_VOL', 'HAI_5_DEN_VOL', 'HAI_6_DEN_VOL', 'HAI_1_DEN_PRED', 'HAI_2_DEN_PRED', 'HAI_3_DEN_PRED', 'HAI_4_DEN_PRED', 'HAI_5_DEN_PRED', 'HAI_6_DEN_PRED', 'HAI_1', 'HAI_2', 'HAI_3', 'HAI_4', 'HAI_5', 'HAI_6', 'READM_30_HOSP_WIDE', 'READM_30_HIP_KNEE', 'EDAC_30_HF', 'READM_30_COPD', 'EDAC_30_AMI', 'EDAC_30_PN', 'MORT_30_STK', 'MORT_30_PN', 'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'READM_30_HOSP_WIDE_DEN', 'READM_30_HIP_KNEE_DEN', 'EDAC_30_HF_DEN', 'READM_30_COPD_DEN', 'EDAC_30_AMI_DEN', 'EDAC_30_PN_DEN', 'MORT_30_STK_DEN', 'MORT_30_PN_DEN', 'MORT_30_HF_DEN', 'MORT_30_COPD_DEN', 'MORT_30_AMI_DEN', 'COMP_HIP_KNEE_DEN', 'OP_2', 'OP_2_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'PSI_4_S

,PROVIDER_ID,HAI_1_DEN_VOL,HAI_2_DEN_VOL,HAI_3_DEN_VOL,HAI_4_DEN_VOL,HAI_5_DEN_VOL,HAI_6_DEN_VOL,HAI_1_DEN_PRED,HAI_2_DEN_PRED,HAI_3_DEN_PRED,HAI_4_DEN_PRED,HAI_5_DEN_PRED,HAI_6_DEN_PRED,HAI_1,HAI_2,HAI_3,HAI_4,HAI_5,HAI_6,READM_30_HOSP_WIDE,READM_30_HIP_KNEE,EDAC_30_HF,READM_30_COPD,EDAC_30_AMI,EDAC_30_PN,MORT_30_STK,MORT_30_PN,MORT_30_HF,MORT_30_COPD,MORT_30_AMI,COMP_HIP_KNEE,READM_30_HOSP_WIDE_DEN,READM_30_HIP_KNEE_DEN,EDAC_30_HF_DEN,READM_30_COPD_DEN,EDAC_30_AMI_DEN,EDAC_30_PN_DEN,MORT_30_STK_DEN,MORT_30_PN_DEN,MORT_30_HF_DEN,MORT_30_COPD_DEN,MORT_30_AMI_DEN,COMP_HIP_KNEE_DEN,OP_2,OP_2_DEN,OP_3B,OP_3B_DEN,OP_8,OP_8_DEN,OP_10,OP_10_DEN,OP_13,OP_13_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,PSI_90_SAFETY,IMM_3_DEN,IMM_3,HCP_COVID_19_DEN,HCP_COVID_19,PC_01,PC_01_DEN,SEP_1,SEP_1_DEN,H_RESP_RATE_P,H_COMP_1_STAR_RATING,H_COMP_2_STAR_RATING,H_COMP_3_STAR_RATING,H_COMP_5_STAR_RATING,H_COMP_6_STAR_RATING,H_COMP_7_STAR_RATING,H_GLOB_STAR_RATING,H_INDI_STAR_RATING,H_NUMB_COMP,PSI_90_SAFETY_DEN,MORT_30_CABG,MORT_30_CABG_DEN,READM_30_CABG,READM_30_CABG_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN
0,010001,10024.0,17731.0,154.0,200.0,101908.0,101451.0,10.597,26.630,4.548,1.845,9.412,72.686,0.661,0.300,1.099,0.0,0.850,0.660,0.142,0.042,21.8,0.199,1.9,-1.5,0.164,0.159,0.083,0.085,0.124,0.024,3058.0,98.0,755.0,202.0,319.0,436.0,489.0,407.0,630.0,182.0,317.0,102.0,NaN,NaN,NaN,NaN,0.425,146.0,0.057,1488.0,0.067,208.0,205.0,323.0,0.03,51079.0,NaN,NaN,0.81,16.0,173.39,120.0,1.01,3795.0,0.97,2323.0,0.737,0.09,34.0,0.46,146.0,15.0,2.0,3.0,2.0,4.0,4.0,3.0,3.0,3.5,434.0,2046.895485,0.047,172.0,0.117,165.0,14.1,254.0,10.2,214.0,4.7,214.0,1.0,688.0
1,010005,3713.0,8670.0,88.0,NaN,38413.0,35686.0,2.450,4.995,2.512,NaN,1.999,10.484,3.673,1.201,1.194,NaN,0.000,0.858,0.139,0.040,9.3,0.176,4.7,2.2,0.166,0.218,0.169,0.081,0.126,0.018,1258.0,178.0,157.0,234.0,38.0,361.0,100.0,369.0,153.0,195.0,52.0,134.0,NaN,NaN,NaN,NaN,0.545,191.0,0.138,1214.0,0.043,208.0,146.0,1003.0,0.03,54503.0,0.73,15.0,0.99,108.0,142.88,35.0,0.91,2593.0,0.90,2026.0,0.821,0.01,194.0,0.59,242.0,16.0,3.0,4.0,2.0,3.0,3.0,3.0,3.0,3.5,717.0,819.043002,NaN,NaN,NaN,NaN,14.6,850.0,11.1,108.0,5.7,108.0,0.9,362.0
2,010006,7318.0,11755.0,91.0,NaN,62709.0,54159.0,7.924,15.296,2.523,NaN,4.164,22.618,0.757,0.196,0.396,NaN,1.441,0.088,0.142,0.048,-2.3,0.177,25.9,42.3,0.189,0.178,0.122,0.078,0.165,0.034,2555.0,246.0,550.0,235.0,312.0,538.0,261.0,528.0,468.0,209.0,295.0,234.0,NaN,NaN,NaN,NaN,0.412,97.0,0.110,1168.0,0.014,217.0,144.0,363.0,0.01,41137.0,0.57,14.0,0.88,75.0,157.42,84.0,1.10,2292.0,0.64,2694.0,0.651,0.00,37.0,0.58,142.0,17.0,2.0,3.0,1.0,2.0,3.0,2.0,2.0,2.5,1358.0,1487.163359,0.035,117.0,0.156,109.0,12.5,1505.0,NaN,NaN,NaN,NaN,1.1,468.0
3,010007,NaN,NaN,NaN,NaN,NaN,5413.0,NaN,NaN,NaN,NaN,NaN,2.148,NaN,NaN,NaN,NaN,NaN,0.466,0.151,NaN,36.3,0.197,NaN,-12.6,NaN,0.217,0.139,0.103,NaN,NaN,272.0,NaN,51.0,72.0,NaN,99.0,NaN,106.0,45.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.059,169.0,NaN,NaN,119.0,1202.0,0.03,11120.0,NaN,NaN,0.63,68.0,NaN,NaN,0.99,318.0,0.61,277.0,0.574,NaN,NaN,0.93,55.0,23.0,3.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,173.0,142.073902,NaN,NaN,NaN,NaN,15.3,118.0,NaN,NaN,NaN,NaN,1.0,56.0
4,010008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.145,NaN,NaN,NaN,NaN,NaN,NaN,0.197,NaN,NaN,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.021,97.0,NaN,NaN,113.0,346.0,0.00,6205.0,NaN,NaN,0.52,23.0,NaN,NaN,NaN,125.0,0.46,169.0,0.623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.3,62.0,NaN,NaN,NaN,NaN,NaN,NaN


## HAIs

In [4]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_01_2024/Healthcare_Associated_Infections-Hospital.csv')
#print(df['Measure ID'].unique())

measures = ['HAI_1_ELIGCASES', 'HAI_1_DOPC', 'HAI_1_SIR', 'HAI_2_ELIGCASES', 'HAI_2_DOPC', 'HAI_2_SIR', 
            'HAI_3_ELIGCASES', 'HAI_3_DOPC', 'HAI_3_SIR', 'HAI_4_ELIGCASES', 'HAI_4_DOPC', 'HAI_4_SIR', 
            'HAI_5_ELIGCASES', 'HAI_5_DOPC', 'HAI_5_SIR', 'HAI_6_ELIGCASES', 'HAI_6_DOPC', 'HAI_6_SIR']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0)

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

hai_df = pd.DataFrame(columns=['Facility ID']) 
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    hai_df = hai_df.merge(tdf2, on='Facility ID', how='outer')
    
hai_df.rename(columns={'HAI_1_ELIGCASES': 'HAI_1_DEN_PRED',
                       'HAI_1_DOPC': 'HAI_1_DEN_VOL',
                       'HAI_1_SIR': 'HAI_1',
                       'HAI_2_ELIGCASES': 'HAI_2_DEN_PRED',
                       'HAI_2_DOPC': 'HAI_2_DEN_VOL',
                       'HAI_2_SIR': 'HAI_2',
                       'HAI_3_ELIGCASES': 'HAI_3_DEN_PRED',
                       'HAI_3_DOPC': 'HAI_3_DEN_VOL',
                       'HAI_3_SIR': 'HAI_3',
                       'HAI_4_ELIGCASES': 'HAI_4_DEN_PRED',
                       'HAI_4_DOPC': 'HAI_4_DEN_VOL',
                       'HAI_4_SIR': 'HAI_4',
                       'HAI_5_ELIGCASES': 'HAI_5_DEN_PRED',
                       'HAI_5_DOPC': 'HAI_5_DEN_VOL',
                       'HAI_5_SIR': 'HAI_5',
                       'HAI_6_ELIGCASES': 'HAI_6_DEN_PRED',
                       'HAI_6_DOPC': 'HAI_6_DEN_VOL',
                       'HAI_6_SIR': 'HAI_6',
                       'Facility ID': 'PROVIDER_ID',
                   }, inplace=True)

for c in list(hai_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass

print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')
hai_df = curate(hai_df)

76 remaining features: ['COMP_HIP_KNEE', 'COMP_HIP_KNEE_DEN', 'EDAC_30_AMI', 'EDAC_30_AMI_DEN', 'EDAC_30_HF', 'EDAC_30_HF_DEN', 'EDAC_30_PN', 'EDAC_30_PN_DEN', 'HCP_COVID_19', 'HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'IMM_3', 'IMM_3_DEN', 'MORT_30_AMI', 'MORT_30_AMI_DEN', 'MORT_30_CABG', 'MORT_30_CABG_DEN', 'MORT_30_COPD', 'MORT_30_COPD_DEN', 'MORT_30_HF', 'MORT_30_HF_DEN', 'MORT_30_PN', 'MORT_30_PN_DEN', 'MORT_30_STK', 'MORT_30_STK_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_2', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_2_DEN', 'OP_32', 'OP_32_DEN', 'OP_35_ADM', 'OP_35_ADM_DEN', 'OP_35_ED', 'OP_35_ED_DEN', 'OP_36', 'OP_36_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DE

## Unplanned Hospital Visits


In [5]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_01_2024/Unplanned_Hospital_Visits-Hospital.csv')
measures = ['EDAC_30_AMI', 'EDAC_30_HF', 'EDAC_30_PN', 'OP_32', 'OP_35_ADM', 'OP_35_ED', 'OP_36', 
            'READM_30_CABG', 'READM_30_COPD', 'READM_30_HIP_KNEE', 'READM_30_HOSP_WIDE']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0)

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Denominator', 'Measure ID', 'Score'], axis=1)

uhv_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    tdf2[m + '_DEN'] = tdf1['Denominator'].tolist()
    uhv_df = uhv_df.merge(tdf2, on='Facility ID', how='outer')

uhv_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)

for c in list(uhv_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass
    
print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')
uhv_df = curate(uhv_df)

54 remaining features: ['COMP_HIP_KNEE', 'COMP_HIP_KNEE_DEN', 'HCP_COVID_19', 'HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'IMM_3', 'IMM_3_DEN', 'MORT_30_AMI', 'MORT_30_AMI_DEN', 'MORT_30_CABG', 'MORT_30_CABG_DEN', 'MORT_30_COPD', 'MORT_30_COPD_DEN', 'MORT_30_HF', 'MORT_30_HF_DEN', 'MORT_30_PN', 'MORT_30_PN_DEN', 'MORT_30_STK', 'MORT_30_STK_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_2', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_2_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DEN', 'PSI_90_SAFETY', 'PSI_90_SAFETY_DEN', 'SEP_1', 'SEP_1_DEN'] 



## COMPLICATIONS AND DEATHS

In [6]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_01_2024/Complications_and_Deaths-Hospital.csv')

measures = ['MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD', 'MORT_30_HF', 
            'MORT_30_PN', 'MORT_30_STK', 'PSI_04', 'COMP_HIP_KNEE',
            'PSI_90']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score', 'Denominator'], axis=1)

cad_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    tdf2[m + '_DEN'] = tdf1['Denominator'].tolist()
    cad_df = cad_df.merge(tdf2, on='Facility ID', how='outer')
    
cad_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'PSI_04': 'PSI_4_SURG_COMP',
                       'PSI_04_DEN': 'PSI_4_SURG_COMP_DEN',
                       'PSI_90': 'PSI_90_SAFETY',
                       'PSI_90_DEN': 'PSI_90_SAFETY_DEN',
                   }, inplace=True)

for c in list(cad_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass
    
print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')
cad_df = curate(cad_df)

36 remaining features: ['HCP_COVID_19', 'HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'IMM_3', 'IMM_3_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_2', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_2_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN', 'SEP_1', 'SEP_1_DEN'] 



## TIMELY AND EFFECTIVE CARE

Everything except PC-01, which for 2023 is located in the Maternal Health files of the hospitals data archive

In [7]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_01_2024/Timely_and_Effective_Care-Hospital.csv',
                encoding='ISO-8859-1')

measures = ['IMM_3', 'OP_18b', 'OP_2', 'OP_22', 'OP_23', 'OP_29', 'OP_3b', 'SEP_1', 'HCP_COVID_19']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Sample', 'Measure ID', 'Score'], axis=1)

tec_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    if m == 'HCP_COVID_19':
        pass
    else:
        tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    
    tec_df = tec_df.merge(tdf2, on='Facility ID', how='outer')
    
tec_df.rename(columns={'Facility ID': 'PROVIDER_ID', 
                       'OP_3b': 'OP_3B', 
                       'OP_3b_DEN': 'OP_3B_DEN', 
                       'OP_18b': 'OP_18B',
                       'OP_18b_DEN': 'OP_18B_DEN',
                      }, inplace=True)

for c in list(tec_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass

print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')
tec_df = curate(tec_df)


19 remaining features: ['HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN'] 



In [8]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_01_2024/VA_TE.csv')

measures = ['IMM-3', 'OP-18b', 'OP-2', 'OP-22', 'OP-23', 'OP-29', 'OP-3b', 'SEP-1', 'HCP-COVID-19']

tec3_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    
    tec3_df = tec3_df.merge(tdf2, on='Facility ID', how='outer')
    
tec3_df.rename(columns={'Facility ID': 'PROVIDER_ID', 
                        'OP-3b': 'OP_3B',
                        'OP-3b_DEN': 'OP_3B_DEN',
                        
                        'OP-18b': 'OP_18B',
                        'OP-18b_DEN': 'OP_18B_DEN',
                        
                        'IMM-3': 'IMM_3', 
                        'IMM-3_DEN': 'IMM_3_DEN', 
                        
                        'OP-2': 'OP_2', 
                        'OP-2_DEN': 'OP_2_DEN', 
                        
                        'OP-22': 'OP_22', 
                        'OP-22_DEN': 'OP_22_DEN', 
                        
                        'OP-23': 'OP_23', 
                        'OP-23_DEN': 'OP_23_DEN', 
                        
                        'OP-29': 'OP_29', 
                        'OP-29_DEN': 'OP_29_DEN', 
                        
                        'SEP-1': 'SEP_1', 
                        'SEP-1_DEN': 'SEP_1_DEN', 
                        
                        'HCP-COVID-19': 'HCP_COVID_19',
                        'HCP-COVID-19_DEN': 'HCP_COVID_19_DEN',
                        
                      }, inplace=True)

tec3_df.replace('Not Available', np.nan, inplace=True)
print(tec3_df.shape)
tec3_df = curate(tec3_df)
print(tec3_df.shape)

print(tec_df.shape)
print(tec3_df.shape)
tec_df = pd.concat([tec_df, tec3_df])
print(tec_df.shape)

tec_df.head()

(137, 19)
(137, 19)
(4675, 18)
(137, 19)
(4812, 19)


,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_2,OP_2_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,OP_3B,OP_3B_DEN,SEP_1,SEP_1_DEN,HCP_COVID_19,HCP_COVID_19_DEN
0,010001,95,3905,214,348,Not Available,Not Available,5,52960,Not Available,Not Available,47,17,Not Available,Not Available,65,127,83.6,NaN
1,010005,80,2700,145,1074,Not Available,Not Available,3,56820,58,12,96,180,57,16,69,252,80.7,NaN
2,010006,67,2536,168,360,Not Available,Not Available,1,42286,75,16,85,82,Not Available,Not Available,57,126,79.6,NaN
3,010007,53,350,132,1275,Not Available,Not Available,4,11202,Not Available,Not Available,23,111,Not Available,Not Available,93,43,60.1,NaN
4,010008,45,126,116,340,Not Available,Not Available,0,6239,Not Available,Not Available,67,24,Not Available,Not Available,Not Available,Not Available,79.7,NaN


## TIMELY AND EFFECTIVE CARE

Only PC-01, which for 2023 is located in the Maternal Health files of the hospitals data archive

In [9]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_01_2024/Maternal_Health-Hospital.csv')
measures = ['PC_01']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score', 'Sample'], axis=1)

tec2_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    tec2_df = tec2_df.merge(tdf2, on='Facility ID', how='outer')
    
tec2_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)

for c in list(tec2_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass

print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')

tec2_df = curate(tec2_df)


17 remaining features: ['HCP_COVID_19_DEN', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_8', 'OP_8_DEN'] 



## HCAHPS

In [10]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_01_2024/HCAHPS-Hospital.csv')

measures = ['H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 
            'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_CLEAN_STAR_RATING',  'H_QUIET_STAR_RATING', 
            'H_RECMND_STAR_RATING', 'H_HSP_RATING_STAR_RATING']

tdf = df[df['HCAHPS Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf.rename(columns={'HCAHPS Measure ID': 'Measure ID'}, inplace=True)
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['HCAHPS Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'HCAHPS Measure ID', 'Patient Survey Star Rating', 
                        'Number of Completed Surveys', 'Survey Response Rate Percent'], axis=1)

HCAHPS_df = pd.DataFrame(columns=['Facility ID'])
for i, m in enumerate(measures):
    tdf1 = df[df['HCAHPS Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Patient Survey Star Rating'].tolist()
    if i == 0:
        tdf2['H_NUMB_COMP'] = tdf1['Number of Completed Surveys'].tolist()
        tdf2['H_RESP_RATE_P'] = tdf1['Survey Response Rate Percent'].tolist()
        
    HCAHPS_df = HCAHPS_df.merge(tdf2, on='Facility ID', how='outer')
    

HCAHPS_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)
HCAHPS_df['H_HSP_RATING_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_HSP_RATING_STAR_RATING'] = HCAHPS_df['H_HSP_RATING_STAR_RATING'].astype(int)
HCAHPS_df['H_RECMND_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_RECMND_STAR_RATING'] = HCAHPS_df['H_RECMND_STAR_RATING'].astype(int)
HCAHPS_df['H_GLOB_STAR_RATING'] = np.round((HCAHPS_df['H_HSP_RATING_STAR_RATING'] + HCAHPS_df['H_RECMND_STAR_RATING']) / 2, 1)
HCAHPS_df['H_GLOB_STAR_RATING'].replace(0, np.nan, inplace=True)

HCAHPS_df['H_CLEAN_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_CLEAN_STAR_RATING'] = HCAHPS_df['H_CLEAN_STAR_RATING'].astype(int)
HCAHPS_df['H_QUIET_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_QUIET_STAR_RATING'] = HCAHPS_df['H_QUIET_STAR_RATING'].astype(int)
HCAHPS_df['H_INDI_STAR_RATING'] = np.round((HCAHPS_df['H_CLEAN_STAR_RATING'] + HCAHPS_df['H_QUIET_STAR_RATING']) / 2, 1)
HCAHPS_df['H_INDI_STAR_RATING'].replace(0, np.nan, inplace=True)

HCAHPS_df.drop(labels = ['H_CLEAN_STAR_RATING',  'H_QUIET_STAR_RATING', 'H_RECMND_STAR_RATING', 'H_HSP_RATING_STAR_RATING'], axis=1, inplace=True)

for c in list(HCAHPS_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass

print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')
HCAHPS_df = curate(HCAHPS_df)

7 remaining features: ['HCP_COVID_19_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_8', 'OP_8_DEN'] 



## Outpatient Imaging Efficiency

The July 2023 Overall Star Ratings were calculated using the measure data from the January 2023 Care Compare update, with the re-released OP-13 measure data publicly reported in April 2023 on Care Compare.

Make two dataframes: 1 for OP-13 (April 2023) and 1 for other measures (Jan 2023)

In [11]:
df = pd.read_csv(stars_dir + 'CareCompare/hospitals_01_2024/Outpatient_Imaging_Efficiency-Hospital.csv')
measures = ['OP-13', 'OP-8', 'OP-10']

tdf = df[df['Measure ID'].isin(measures + ['End Date', 'Start Date'])]
tdf = tdf.filter(items = ['Measure ID', 'Start Date', 'End Date'])
tdf.drop_duplicates(inplace=True)
dates_df = pd.concat([dates_df, tdf], axis=0, ignore_index=True)


df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

oie_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    oie_df = oie_df.merge(tdf2, on='Facility ID', how='outer')
    
    
oie_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'OP-13': 'OP_13',
                       'OP-8': 'OP_8',
                       'OP-10': 'OP_10',
                   }, inplace=True)

for c in list(oie_df):
    try:
        sas_cols_2023.remove(c)
    except:
        pass

print(len(sas_cols_2023), 'remaining features:', sorted(sas_cols_2023), '\n')
oie_df1 = curate(oie_df)


4 remaining features: ['HCP_COVID_19_DEN', 'OP_10_DEN', 'OP_13_DEN', 'OP_8_DEN'] 



## MERGE DATAFRAME AND COMPARE TO SAS FILE

In [12]:
print(dates_df.shape)
dates_df.to_csv(stars_dir + "2024/Jan_2024_measure_dates.csv", index=False)

(61, 3)


In [13]:
main_df = tec_df.merge(tec2_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(cad_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(HCAHPS_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(uhv_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(hai_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(oie_df1, on='PROVIDER_ID', how='outer')

main_df['OP_10_DEN'] = np.nan
main_df['OP_13_DEN'] = np.nan
main_df['OP_8_DEN'] = np.nan
main_df['HCP_COVID_19_DEN'] = np.nan

sas_cols2 = list(sas_cols)
for c in list(main_df):
    try:
        sas_cols2.remove(c)
    except:
        pass

print(len(sas_cols2), 'remaining features:', sorted(sas_cols2), '\n')
print(main_df.shape)

for col in list(main_df):
    if col != 'PROVIDER_ID':
        main_df[col] = pd.to_numeric(main_df[col], errors='coerce')


0 remaining features: [] 

(4812, 95)


In [14]:
tdf = main_df.copy(deep=True)

# Remove children's hospitals
prvdrs1 = tdf['PROVIDER_ID'].unique().tolist()
prvdrs2 = []
for p in prvdrs1:
    
    try:
        p_suf = float(p[2:])

        if p_suf < 3300 or p_suf > 3399:
            prvdrs2.append(p)
            
    except:
        prvdrs2.append(p)
        
tdf = tdf[tdf['PROVIDER_ID'].isin(prvdrs2)]

# Remove hospitals containing no data for overall star rating measures
tdf = tdf.loc[:, ~tdf.columns.str.contains('_DEN')]
ls = list(tdf)
ls.remove('PROVIDER_ID')
tdf.dropna(how='all', axis=0, subset=ls, inplace=True)

main_df = main_df[main_df['PROVIDER_ID'].isin(tdf['PROVIDER_ID'].unique().tolist())]
print('main_df.shape:', main_df.shape)

'''
# Remove select Govt/Military-based hospitals
ls = ['02013F', '02014F', '05015F', '05020F', '05022F', '05039F', 
       '05041F', '06003F', '10013F', '10021F', '11032F', '11033F', 
       '11035F', '12001F', '17002F', '18003F', '19050F', '21007F', 
       '25039F', '26002F', '29001F', '33025F', '34011F', '34014F', 
       '36006F', '42009F', '45068F', '45069F', '45070F', '49001F', 
       '49005F', '49008F', '50005F', '50006F']
main_df = main_df[~main_df['PROVIDER_ID'].isin(ls)]
'''

ls = np.setdiff1d(list(main_df), sas_cols)
main_df = main_df.filter(items=sas_cols, axis=1)
print('main_df.shape:', main_df.shape)


main_df.shape: (4654, 95)
main_df.shape: (4654, 95)


## Multiple select features by 0.01

Several features in the SAS file are represented as rates on the scale of 0 to 1, but are represented in the hospitals data archive as percentages on the scale of 0 to 100.

In [15]:
ls = list(main_df)
ls.remove('PROVIDER_ID')

prvdrs_main1 = sorted(main_df['PROVIDER_ID'].tolist())

ls = ['READM_30_HIP_KNEE', 'READM_30_COPD', 'MORT_30_STK', 'MORT_30_PN',
      'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'OP_22',
      'OP_23', 'OP_29', 'IMM_3', 'PC_01', 'SEP_1', 'MORT_30_CABG',
      'READM_30_CABG', 'READM_30_HOSP_WIDE', 'OP_2', 'OP_8',
      'OP_10', 'OP_13', 'HCP_COVID_19']

for l in ls: 
    main_df[l] = main_df[l] * 0.01

## Prepare data for import into SAS as csv

**Issue:** Python can read SAS files but it can't write them. Only SAS can write SAS files. Consequently, the dataframe produced here must be exported as a CSV file. When importing the CSV into SAS, SAS has a problem interpreting the CMS IDs of hospitals because it tries to interpret some IDs as numbers and others as strings (character variables). This is because VHA hospitals contain an 'F' at the end of their IDs.

**Solution:** Remove the 'F' suffix from VHA hospitals and replaced it with 666666. Doing so alleviate the issue of importing the CSV file and not causing in error in reading CMS IDs, while preventing ID conflicts.

In [16]:
prvdrs = []
for p in main_df['PROVIDER_ID'].tolist():
    if 'F' in p:
        p = p[:-1]
        p = p + '666666'
    prvdrs.append(p)

main_df['PROVIDER_ID'] = prvdrs

print(list(main_df))
main_df.head()


['PROVIDER_ID', 'HAI_1_DEN_VOL', 'HAI_2_DEN_VOL', 'HAI_3_DEN_VOL', 'HAI_4_DEN_VOL', 'HAI_5_DEN_VOL', 'HAI_6_DEN_VOL', 'HAI_1_DEN_PRED', 'HAI_2_DEN_PRED', 'HAI_3_DEN_PRED', 'HAI_4_DEN_PRED', 'HAI_5_DEN_PRED', 'HAI_6_DEN_PRED', 'HAI_1', 'HAI_2', 'HAI_3', 'HAI_4', 'HAI_5', 'HAI_6', 'READM_30_HOSP_WIDE', 'READM_30_HIP_KNEE', 'EDAC_30_HF', 'READM_30_COPD', 'EDAC_30_AMI', 'EDAC_30_PN', 'MORT_30_STK', 'MORT_30_PN', 'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'READM_30_HOSP_WIDE_DEN', 'READM_30_HIP_KNEE_DEN', 'EDAC_30_HF_DEN', 'READM_30_COPD_DEN', 'EDAC_30_AMI_DEN', 'EDAC_30_PN_DEN', 'MORT_30_STK_DEN', 'MORT_30_PN_DEN', 'MORT_30_HF_DEN', 'MORT_30_COPD_DEN', 'MORT_30_AMI_DEN', 'COMP_HIP_KNEE_DEN', 'OP_2', 'OP_2_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DEN', 'PSI_90_SAFETY', 'IMM_3_DEN', 'IMM_3', 'HCP_CO

,PROVIDER_ID,HAI_1_DEN_VOL,HAI_2_DEN_VOL,HAI_3_DEN_VOL,HAI_4_DEN_VOL,HAI_5_DEN_VOL,HAI_6_DEN_VOL,HAI_1_DEN_PRED,HAI_2_DEN_PRED,HAI_3_DEN_PRED,HAI_4_DEN_PRED,HAI_5_DEN_PRED,HAI_6_DEN_PRED,HAI_1,HAI_2,HAI_3,HAI_4,HAI_5,HAI_6,READM_30_HOSP_WIDE,READM_30_HIP_KNEE,EDAC_30_HF,READM_30_COPD,EDAC_30_AMI,EDAC_30_PN,MORT_30_STK,MORT_30_PN,MORT_30_HF,MORT_30_COPD,MORT_30_AMI,COMP_HIP_KNEE,READM_30_HOSP_WIDE_DEN,READM_30_HIP_KNEE_DEN,EDAC_30_HF_DEN,READM_30_COPD_DEN,EDAC_30_AMI_DEN,EDAC_30_PN_DEN,MORT_30_STK_DEN,MORT_30_PN_DEN,MORT_30_HF_DEN,MORT_30_COPD_DEN,MORT_30_AMI_DEN,COMP_HIP_KNEE_DEN,OP_2,OP_2_DEN,OP_3B,OP_3B_DEN,OP_8,OP_8_DEN,OP_10,OP_10_DEN,OP_13,OP_13_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,PSI_90_SAFETY,IMM_3_DEN,IMM_3,HCP_COVID_19_DEN,HCP_COVID_19,PC_01,PC_01_DEN,SEP_1,SEP_1_DEN,H_RESP_RATE_P,H_COMP_1_STAR_RATING,H_COMP_2_STAR_RATING,H_COMP_3_STAR_RATING,H_COMP_5_STAR_RATING,H_COMP_6_STAR_RATING,H_COMP_7_STAR_RATING,H_GLOB_STAR_RATING,H_INDI_STAR_RATING,H_NUMB_COMP,PSI_90_SAFETY_DEN,MORT_30_CABG,MORT_30_CABG_DEN,READM_30_CABG,READM_30_CABG_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN
0,010001,9149.0,17310.0,214.0,76.0,104733.0,104733.0,9.597,24.766,5.994,0.712,11.400,67.066,0.938,0.363,1.335,NaN,0.965,0.507,0.142,0.038,23.4,0.190,-15.4,23.6,0.148,0.180,0.089,0.088,0.120,0.027,2912.0,49.0,614.0,117.0,274.0,403.0,398.0,400.0,549.0,107.0,278.0,49.0,NaN,NaN,NaN,NaN,0.380,NaN,0.061,NaN,0.028,NaN,214.0,348.0,0.05,52960.0,NaN,NaN,0.47,17.0,184.68,134.0,1.21,3905.0,0.95,NaN,0.836,0.00,32.0,0.65,127.0,15.0,3.0,3.0,3.0,3.0,4.0,4.0,3.5,3.5,544.0,NaN,0.041,132.0,0.105,126.0,12.9,170.0,11.9,202.0,4.9,202.0,1.1,668.0
1,010005,3194.0,8277.0,96.0,34.0,36794.0,34887.0,1.989,4.019,2.626,0.324,1.847,10.066,2.514,0.995,0.762,NaN,0.541,0.497,0.128,0.034,22.1,0.166,NaN,-6.2,0.153,0.233,0.149,0.099,0.136,0.023,1052.0,172.0,129.0,136.0,NaN,285.0,81.0,289.0,121.0,126.0,27.0,155.0,NaN,NaN,57.0,16.0,0.477,NaN,0.120,NaN,0.042,NaN,145.0,1074.0,0.03,56820.0,0.58,12.0,0.96,180.0,183.49,43.0,0.97,2700.0,0.80,NaN,0.807,0.02,200.0,0.69,252.0,18.0,3.0,4.0,1.0,3.0,4.0,3.0,3.0,3.0,824.0,NaN,NaN,NaN,NaN,NaN,14.2,739.0,7.9,107.0,5.5,107.0,1.9,406.0
2,010006,5343.0,8715.0,111.0,14.0,63727.0,60304.0,5.801,11.166,2.950,0.124,5.283,27.805,0.172,0.358,0.000,NaN,1.514,0.072,0.134,0.053,-4.7,0.176,28.1,-0.4,0.172,0.195,0.125,0.099,0.165,0.046,2310.0,138.0,441.0,158.0,273.0,472.0,227.0,469.0,388.0,148.0,254.0,145.0,NaN,NaN,NaN,NaN,0.462,NaN,0.101,NaN,0.045,NaN,168.0,360.0,0.01,42286.0,0.75,16.0,0.85,82.0,173.63,96.0,1.17,2536.0,0.67,NaN,0.796,0.04,28.0,0.57,126.0,19.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,2.5,1503.0,NaN,0.036,95.0,0.124,89.0,12.1,1355.0,NaN,NaN,NaN,NaN,1.4,484.0
3,010007,343.0,1193.0,3.0,NaN,5511.0,5511.0,0.210,0.652,0.089,NaN,0.133,2.660,NaN,NaN,NaN,NaN,NaN,0.376,0.157,0.042,-1.9,0.200,NaN,-9.4,NaN,0.285,0.125,0.137,NaN,NaN,258.0,26.0,31.0,34.0,NaN,72.0,NaN,88.0,26.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.034,NaN,NaN,NaN,132.0,1275.0,0.04,11202.0,NaN,NaN,0.23,111.0,NaN,NaN,0.95,350.0,0.53,NaN,0.601,NaN,NaN,0.93,43.0,24.0,3.0,5.0,4.0,3.0,3.0,3.0,3.5,3.0,189.0,NaN,NaN,NaN,NaN,NaN,13.4,109.0,NaN,NaN,NaN,NaN,1.2,59.0
4,010008,41.0,444.0,NaN,NaN,2376.0,2376.0,0.023,0.242,NaN,NaN,0.055,0.436,NaN,NaN,NaN,NaN,NaN,NaN,0.148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,116.0,340.0,0.00,6239.0,NaN,NaN,0.67,24.0,NaN,NaN,NaN,126.0,0.45,NaN,0.797,NaN,NaN,NaN,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,NaN,NaN,NaN,NaN,NaN,12.9,42.0,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
main_df['PROVIDER_ID'] = pd.to_numeric(main_df['PROVIDER_ID'], errors='coerce')
main_df.sort_values(by=['PROVIDER_ID'], ascending = True, inplace = True)
main_df.to_csv(stars_dir + "Reproduce_Stars_Input/2024/Input_File/data_for_2024_prognostications_from_Jan2024.csv", index=False)
